In [22]:
import os
import json
import pandas as pd
from typing import Dict, Any
from urllib.request import urlretrieve
import seaborn as sns
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
from shapely import wkt
import openrouteservice

relative_path = "../data/landing/"
absolute_path = os.path.abspath(relative_path)

In [23]:
pwd

'/Users/manansaddi/Desktop/ads project 2/project-2-group-real-estate-industry-project-32/notebooks'

In [24]:
# load the parquet file called domain_data.parquet
df = pd.read_csv(relative_path+ "S2times.csv")

In [25]:
df

,FID,sa22016id,sa22016pid,sa22016name,sa22016code,sa22016code5digit,state,dateyear,datemonth,enddate,...,sold_both_auction_private_treaty_detailedposition70price,sold_both_auction_private_treaty_detailedposition75price,sold_both_auction_private_treaty_detailedposition80price,sold_both_auction_private_treaty_detailedposition85price,sold_both_auction_private_treaty_detailedposition90price,sold_both_auction_private_treaty_detailedposition95price,sold_both_auction_private_treaty_standarddeviationprice,sold_both_auction_private_treaty_geometricmeanprice,gross_rental_yield,gid
0,apm_sa2_2016_timeseries_table.999795,1283,SA2_16319011500,Millbank - Avoca,319011500,31500,QLD,2013,1,31/01/2013 12:00:00 AM,...,328500.0,345000.0,351000.0,383000.0,430000.0,467000.0,93903.35,294551.7,0.052000,999795
1,apm_sa2_2016_timeseries_table.999796,1283,SA2_16319011500,Millbank - Avoca,319011500,31500,QLD,2014,1,31/01/2014 12:00:00 AM,...,330500.0,336000.0,342000.0,359000.0,399000.0,499000.0,75379.23,299861.3,0.052881,999796
2,apm_sa2_2016_timeseries_table.999797,1283,SA2_16319011500,Millbank - Avoca,319011500,31500,QLD,2015,1,31/01/2015 12:00:00 AM,...,329000.0,337000.0,349000.0,365000.0,390000.0,460000.0,110069.50,299050.3,0.052703,999797
3,apm_sa2_2016_timeseries_table.999798,1283,SA2_16319011500,Millbank - Avoca,319011500,31500,QLD,2016,1,31/01/2016 12:00:00 AM,...,330000.0,335000.0,356000.0,376000.0,395000.0,420000.0,83685.41,285904.3,0.056811,999798
4,apm_sa2_2016_timeseries_table.999799,1283,SA2_16319011500,Millbank - Avoca,319011500,31500,QLD,2017,1,31/01/2017 12:00:00 AM,...,333000.0,343000.0,369000.0,380000.0,410000.0,439000.0,96371.87,286405.6,0.058886,999799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,apm_sa2_2016_timeseries_table.503944,1998,SA2_16214021384,Rosebud - McCrae,214021384,21384,VIC,2013,2,28/02/2013 12:00:00 AM,...,399950.0,405000.0,420000.0,460000.0,470000.0,550000.0,103339.50,354320.1,0.040114,503944
999996,apm_sa2_2016_timeseries_table.503945,1998,SA2_16214021384,Rosebud - McCrae,214021384,21384,VIC,2014,2,28/02/2014 12:00:00 AM,...,445000.0,445000.0,457500.0,500000.0,620000.0,865000.0,161944.10,377546.7,0.037824,503945
999997,apm_sa2_2016_timeseries_table.503986,1688,SA2_16205051102,Rosedale,205051102,21102,VIC,2018,2,28/02/2018 12:00:00 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,503986
999998,apm_sa2_2016_timeseries_table.503946,1998,SA2_16214021384,Rosebud - McCrae,214021384,21384,VIC,2015,2,28/02/2015 12:00:00 AM,...,400000.0,445000.0,470000.0,530000.0,578000.0,600000.0,143132.40,359917.8,0.042092,503946


In [26]:
# find number of unique values in each column
df_vic = df[df['state'] == 'VIC']

In [27]:
df_vic
# count nan values in each column sorted by the number of nan values
df_vic.isna().sum().sort_values(ascending=False)

sold_private_treaty_averagediscount            132738
auction_activity_auctionclearancerate          121183
gross_rental_yield                             101413
sold_private_treaty_averagedaysonmarket         99120
for_rent_home_lease_detailedposition05price     95179
                                                ...  
sa22016code5digit                                   0
sa22016code                                         0
sa22016name                                         0
sa22016pid                                          0
gid                                                 0
Length: 120, dtype: int64

In [28]:
# list of columns 
list(df_vic.columns)
rent_columns = []
for col in df_vic.columns:
    if('rental' in col.lower() or 'rent' in col.lower() and 'price' in col.lower()):
        rent_columns.append(col)

In [29]:
print(rent_columns)

['for_rent_home_lease_pricecalcreccount', 'for_rent_home_lease_averageprice', 'for_rent_home_lease_maximumprice', 'for_rent_home_lease_minimumprice', 'for_rent_home_lease_totalprice', 'for_rent_home_lease_medianprice', 'for_rent_home_lease_position25price', 'for_rent_home_lease_position75price', 'for_rent_home_lease_detailedpricecalcreccount', 'for_rent_home_lease_detailedposition05price', 'for_rent_home_lease_detailedposition10price', 'for_rent_home_lease_detailedposition15price', 'for_rent_home_lease_detailedposition20price', 'for_rent_home_lease_detailedposition25price', 'for_rent_home_lease_detailedposition30price', 'for_rent_home_lease_detailedposition35price', 'for_rent_home_lease_detailedposition40price', 'for_rent_home_lease_detailedposition45price', 'for_rent_home_lease_detailedposition50price', 'for_rent_home_lease_detailedposition55price', 'for_rent_home_lease_detailedposition60price', 'for_rent_home_lease_detailedposition65price', 'for_rent_home_lease_detailedposition70pric

In [30]:
df_important_columns = df_vic[['dateyear','for_rent_home_lease_medianprice','for_rent_home_lease_averageprice',
 'for_rent_home_lease_maximumprice',
 'for_rent_home_lease_minimumprice',
 'for_rent_home_lease_totalprice','for_rent_home_lease_geometricmeanprice','sa22016name','sa22016code']]

In [31]:
df_important_columns.isna().sum().sort_values(ascending=False)
# len(df_important_columns)

for_rent_home_lease_medianprice           95179
for_rent_home_lease_averageprice          75016
for_rent_home_lease_maximumprice          75016
for_rent_home_lease_minimumprice          75016
for_rent_home_lease_totalprice            75016
for_rent_home_lease_geometricmeanprice    75016
dateyear                                      0
sa22016name                                   0
sa22016code                                   0
dtype: int64

In [35]:
ndf_important_columns['sa22016code'].unique()

array([204021064, 201011007, 205031088, 210011227, 205031087, 201031017,
       203021044, 204011054, 205051099, 206071139, 210011226, 201021009,
       201011001, 206051128, 205051100, 206021110, 213021341, 203011034,
       215011387, 213021342, 213021343, 213011329, 212011287, 205021081,
       215011386, 213011328, 206031113, 205041094, 206061135, 208031183,
       208031184, 212051319, 202031032, 207011146, 201031013, 213041353,
       201011002, 212011288, 201011003, 201031014, 202021026, 201011004,
       211011251, 207011147, 203021037, 209031211, 207011148, 216011408,
       211031263, 208011168, 204031068, 211051274, 204021063, 202011018,
       208021174, 207031161, 212021293, 208021427, 208021426, 202021028,
       217031473, 212031306, 213041461, 212021294, 207031162, 205021082,
       211011446, 213041462, 207031163, 207031164, 213031347, 204031069,
       210051242, 208011169, 202011023, 208011170, 210051442, 206011105,
       206011106, 206011107, 201021011, 207021156, 

In [32]:
df_important_columns_no = df_important_columns.dropna()

In [34]:
df_important_columns_no.isna().sum().sort_values(ascending=False) 
len(df_important_columns_no)  

120099

In [13]:
# forcast the average rental price for the next 5 years
# find the average rental price for each year
# df_vic['year'] = pd.to_datetime(df_vic['date']).dt.year
# df_vic['month'] = pd.to_datetime(df_vic['date']).dt.month
# df_vic['day'] = pd.to_datetime(df_vic['date']).dt.day
# df_vic['dayofweek'] = pd.to_datetime(df_vic['date']).dt.dayofweek
# df_vic['dayofyear'] = pd.to_datetime(df_vic['date']).dt.dayofyear
# df_vic['weekofyear'] = pd.to_datetime(df_vic['date']).dt.weekofyear
# df_vic['quarter'] = pd.to_datetime(df_vic['date']).dt.quarter
# df_vic['year_month'] = pd.to_datetime(df_vic['date']).dt.to_period('M')
# df_vic['year_week'] = pd.to_datetime(df_vic['date']).dt.to_period('W')
# df_vic['year_quarter'] = pd.to_datetime(df_vic['date']).dt.to_period('Q')

KeyError: 'date'